In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install -q datasets

     |████████████████████████████████| 362 kB 7.6 MB/s 
     |████████████████████████████████| 1.1 MB 45.3 MB/s 
     |████████████████████████████████| 140 kB 48.3 MB/s 
     |████████████████████████████████| 212 kB 52.0 MB/s 
     |████████████████████████████████| 101 kB 7.5 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
     |████████████████████████████████| 127 kB 48.7 MB/s 
     |████████████████████████████████| 271 kB 45.6 MB/s 
     |████████████████████████████████| 144 kB 46.6 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
! pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.3 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 6.6 MB 44.3 MB/s 


In [5]:
! pip install torch==1.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 748.8 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.6.0 which is incompatible.
fastai 2.6.3 requires torch<1.12,>=1.7.0, but you have torch 1.6.0 which is incompatible.


In [6]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/train.csv')
eval_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/test.csv')

## Score meanings
The scores are in the 0-1 range with increments of 0.25 with the following meanings:
```
1.0 - Very close match. This is typically an exact match except possibly for differences in conjugation, quantity (e.g. singular vs. plural), and addition or removal of stopwords (e.g. “the”, “and”, “or”).
0.75 - Close synonym, e.g. “mobile phone” vs. “cellphone”. This also includes abbreviations, e.g. "TCP" -> "transmission control protocol".
0.5 - Synonyms which don’t have the same meaning (same function, same properties). This includes broad-narrow (hyponym) and narrow-broad (hypernym) matches.
0.25 - Somewhat related, e.g. the two phrases are in the same high level domain but are not synonyms. This also includes antonyms.
0.0 - Unrelated.
```

## Columns



*   id - a unique identifier for a pair of phrases
*   anchor - the first phrase
*   target - the second phrase
*   context - the CPC classification (version 2021.05), which indicates the subject 
*   within which the similarity is to be scored
*   score - the similarity. This is sourced from a combination of one or more 
manual expert ratings.





In [8]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [9]:
eval_df

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04
5,474c874d0c07bd21,dry corn,dry corn starch,C12
6,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11
7,b8ae62ea5e1d8bdb,angular contact bearing,contact therapy radiation,B23
8,faaddaf8fcba8a3f,produce liquid hydrocarbons,produce a treated stream,C10
9,ae0262c02566d2ce,diesel fuel tank,diesel fuel tanks,F02


*include*：这个参数默认是只计算数值型特征的统计量，当输入include=['O']，会计算离散型变量的统计特征

In [10]:
df.describe(include='O')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


We can see that in the 36473 rows, there are 733 unique anchors, 106 contexts, and nearly 30000 targets. Some anchors are very common, with "component composite coating" for instance appearing 152 times.

In [11]:
df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
df.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1:A47; TEXT2:abatement of pollution; ANC1:...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1:A47; TEXT2:act of abating; ANC1:abatement
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1:A47; TEXT2:active catalyst; ANC1:abatement
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1:A47; TEXT2:eliminating process; ANC1:aba...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1:A47; TEXT2:forest region; ANC1:abatement


## datasets Dataset类
### 构造方法
```
函数原型
datasets.Dataset(
		arrow_table: Table,
        info: Optional[DatasetInfo] = None,
        split: Optional[NamedSplit] = None,
        indices_table: Optional[Table] = None,
        fingerprint: Optional[str] = None,
    )
函数说明
根据ArrowTable产生一个Dataset对象。
```
**类属性/方法**\
1、from_dict函数\
根据字典来创建Dataset数据集对象。\
2、from_pandas函数\
根据pandas中的DataFrame对象创建Dataset对象。\
3、from_csv函数\
根据csv文件来创建Dataset对象。\
4、from_json函数\
5、from_text函数\
....

**对象属性/方法**
1、data属性\
获取Dataset中的数据。

2、cache_files属性\
获取Dataset中数据所在的缓存文件。

3、num_columns、num_rows属性\
获取Dataset中数据的列数、行数。

4、column_names属性\
获取Dataset中的列名称。

5、shape属性\
获取Dataset中的数据形状。

6、unique函数\
在指定列中返回一个不重复的列表。

7、add_column函数\
在Dataset数据集中增加一列数据。

8、add_item函数\
在Dataset数据集中增加一行数据。

9、cast函数\
将Dataset中特征features装换成新的features。

10、cast_column函数\
转换features中的某一列。


## DatasetDict
Dictionary with split names as keys (‘train’, ‘test’ for example), and Dataset objects as values. It also has dataset transform methods like map or filter, to process all the splits at once.

以拆分名称作为键（例如“train”、“test”）和 Dataset 对象作为值的字典。它还具有数据集转换方法，例如 map 或 filter，可以一次处理所有拆分。

### map
Apply a function to all the elements in the table (individually or in batches) and update the table (if function does updated examples). The transformation is applied to all the datasets of the dataset dictionary.

将函数应用于表中的所有元素（单独或批量）并更新表（如果函数确实更新了示例）。转换应用于数据集字典的所有数据集。
```
**function** (callable) — with one of the following signature:
  function(example: Dict[str, Any]) -> Dict[str, Any] if batched=False and with_indices=False
  function(example: Dict[str, Any], indices: int) -> Dict[str, Any] if batched=False and with_indices=True
  function(batch: Dict[str, List]) -> Dict[str, List] if batched=True and with_indices=False
  function(batch: Dict[str, List], indices: List[int]) -> Dict[str, List] if batched=True and with_indices=True

For advanced usage, the function can also return a pyarrow.Table. Moreover if your function returns nothing (None), then map will run your function and return the dataset unchanged.

**batched** (bool, defaults to False) — Provide batch of examples to function
```

In [12]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [13]:
from transformers import BertTokenizer
import torch
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model')

In [14]:
tokenizer.tokenize("A platypus is an ornithorhynchus anatinus.")

['a',
 'p',
 '##la',
 '##ty',
 '##pus',
 'is',
 'an',
 'or',
 '##ni',
 '##th',
 '##or',
 '##hy',
 '##nch',
 '##us',
 'an',
 '##at',
 '##in',
 '##us',
 '.']

In [15]:
tokenizer("A platypus is an ornithorhynchus anatinus.")

{'input_ids': [101, 143, 158, 8461, 8661, 12657, 8310, 9064, 8549, 8833, 8414, 8372, 9943, 11106, 8293, 9064, 8488, 8277, 8293, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
def tok_func(x): return tokenizer(x['input'])

In [17]:
tok_ds = ds.map(tok_func, batched = True)

Parameter 'function'=<function tok_func at 0x7fb8daeedb00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/37 [00:00<?, ?ba/s]

In [18]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [19]:
tok_ds[0]

{'anchor': 'abatement',
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'context': 'A47',
 'id': '37d61fd2272659b1',
 'input': 'TEXT1:A47; TEXT2:abatement of pollution; ANC1:abatement',
 'input_ids': [101,
  10539,
  8148,
  131,
  9367,
  8161,
  132,
  10539,
  8144,
  131,
  9386,
  9690,
  8631,
  8205,
  8559,
  8390,
  9194,
  8410,
  132,
  9064,
  8177,
  8148,
  131,
  9386,
  9690,
  8631,
  102],
 'score': 0.5,
 'target': 'abatement of pollution',
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

## Test and validation sets

In [20]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


This is the test set. Possibly the most important idea in machine learning is that of having separate training, validation, and test data sets.

The validation set is only ever used to see how we're doing. It's never used as inputs to training the model.

Transformers uses a DatasetDict for holding your training and validation sets. To create one that contains 25% of our data for the validation set, and 75% for the training set, use train_test_split:

In [24]:
from numpy.random import normal,seed,uniform
np.random.seed(2022)
train_and_val = tok_ds.train_test_split(0.25, seed = 2022)
train_and_val

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

As you see above, the validation set here is called test and not validate

In practice, a random split like we've used here might not be a good idea -- here's what Dr Rachel Thomas has to say about it:
```
"One of the most likely culprits for this disconnect between results in development vs results in production is a poorly chosen validation set (or even worse, no validation set at all). Depending on the nature of your data, choosing a validation set can be the most important step. Although sklearn offers a train_test_split method, this method takes a random subset of the data, which is a poor choice for many real-world problems."
开发结果与生产结果之间的这种脱节最可能的罪魁祸首之一是选择不当的验证集（或者更糟糕的是，根本没有验证集）。 根据数据的性质，选择验证集可能是最重要的一步。 尽管 sklearn 提供了一个 train_test_split 方法，但该方法采用数据的随机子集，这对于许多现实世界的问题来说是一个糟糕的选择。
```

In [25]:
eval_df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})